# Pokemon Image Classification

### Pokémon TCG API

In [6]:
# Load .env file for Pokémon TCG API
from dotenv import load_dotenv
from pokemontcgsdk import Card
from pokemontcgsdk import Set
from pokemontcgsdk import Type
from pokemontcgsdk import Supertype
from pokemontcgsdk import Subtype
from pokemontcgsdk import Rarity
import json

load_dotenv()

cards = Card.all()

# Convert the list of card objects to a list of dictionaries
cards_dict = [card.__dict__ for card in cards]

# Custom serialization function
def custom_serializer(obj):
    if hasattr(obj, '__dict__'):
        return obj.__dict__
    else:
        return str(obj)

# Open the file in write mode and dump the JSON data
with open("data/all_cards.json", "w") as file:
    json.dump(cards_dict, file, default=custom_serializer)

### Image Downloads

In [1]:
import json

# Step 1: Read the JSON file
with open('data/all_cards.json', 'r') as file:
    data = json.load(file)

# Step 2: Extract URLs of images
image_urls = []
for card in data:
    if 'images' in card:
        if 'small' in card['images']:
            image_urls.append(card['images']['small'])
        if 'large' in card['images']:
            image_urls.append(card['images']['large'])

# Step 3: Optionally, save the URLs to a new file
with open('data/image_urls.txt', 'w') as file:
    for url in image_urls:
        file.write(url + '\n')

# Print the extracted URLs
print(image_urls)

['https://images.pokemontcg.io/hgss4/1.png', 'https://images.pokemontcg.io/hgss4/1_hires.png', 'https://images.pokemontcg.io/xy5/1.png', 'https://images.pokemontcg.io/xy5/1_hires.png', 'https://images.pokemontcg.io/pl1/1.png', 'https://images.pokemontcg.io/pl1/1_hires.png', 'https://images.pokemontcg.io/dp3/1.png', 'https://images.pokemontcg.io/dp3/1_hires.png', 'https://images.pokemontcg.io/det1/1.png', 'https://images.pokemontcg.io/det1/1_hires.png', 'https://images.pokemontcg.io/dv1/1.png', 'https://images.pokemontcg.io/dv1/1_hires.png', 'https://images.pokemontcg.io/mcd19/1.png', 'https://images.pokemontcg.io/mcd19/1_hires.png', 'https://images.pokemontcg.io/pl3/1.png', 'https://images.pokemontcg.io/pl3/1_hires.png', 'https://images.pokemontcg.io/ex12/1.png', 'https://images.pokemontcg.io/ex12/1_hires.png', 'https://images.pokemontcg.io/ex3/1.png', 'https://images.pokemontcg.io/ex3/1_hires.png', 'https://images.pokemontcg.io/ru1/1.png', 'https://images.pokemontcg.io/ru1/1_hires.png

In [10]:
from PIL import Image, ImageEnhance, ImageFilter
import pytesseract

# Set the Tesseract executable path
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

# Define a function to preprocess and extract text from an image
def preprocess_image(image):
    # Convert to grayscale
    image = image.convert('L')
    # Increase contrast
    enhancer = ImageEnhance.Contrast(image)
    image = enhancer.enhance(2)
    # Apply thresholding
    image = image.point(lambda p: p > 128 and 255)
    return image

# Define a function to extract text from a cropped image
def extract_pokemon_name(image_path):
    try:
        # Open the image
        image = Image.open(image_path)
        
        # Crop the image to the region where the Pokémon name is likely located
        width, height = image.size
        cropped_image = image.crop((int(width * 0.1), int(height * 0.05), int(width * 0.9), int(height * 0.2)))  # Adjust the cropping region as needed
        
        # Preprocess the image
        processed_image = preprocess_image(cropped_image)
        
        # Use Tesseract to do OCR on the image with specific configuration
        custom_config = r'--oem 3 --psm 6'
        text = pytesseract.image_to_string(processed_image, config=custom_config)

        # Split the text into lines and find the line with the name
        lines = text.split('\n')
        for line in lines:
            if 'HP' in line:  # The name is usually above the 'HP' text
                name_index = lines.index(line) - 1
                if name_index >= 0:
                    return lines[name_index].strip()
        return "Name not found"
    except Exception as e:
        return str(e)

# Paths to the images
image_path_1 = "./output/card_one_1.jpg"
image_path_2 = "./output/card_one_2.jpg"

# Extract Pokémon names from the images
pokemon_name_1 = extract_pokemon_name(image_path_1)
pokemon_name_2 = extract_pokemon_name(image_path_2)

# Print the extracted names
print("Pokémon name from first card:", pokemon_name_1)
print("Pokémon name from second card:", pokemon_name_2)


Pokémon name from first card: Name not found
Pokémon name from second card: Name not found
